In [14]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

model_name = 'Qwen/Qwen2-VL-2B-Instruct'
model_name = 'Qwen/Qwen2-VL-7B-Instruct'

# default: Load the model on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name, torch_dtype="auto", device_map="auto"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2-VL-2B-Instruct",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

# default processer
processor = AutoProcessor.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

c:\users\yushl\desktop\ws\qwen\env\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yushl\.cache\huggingface\hub\models--Qwen--Qwen2-VL-7B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [ ]:
%%time
# The default range for the number of visual tokens per image in the model is 4-16384. You can set min_pixels and max_pixels according to your needs, such as a token count range of 256-1280, to balance speed and memory usage.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "invoice7.png",
            },
            {"type": "text", "text": "extract values from image along with bounding boxes in a JSON format"},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
#inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=1024)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)


In [ ]:
import json

j = json.loads('''

{\n  "current_assets": {\n    "Cash": 55300,\n    "Marketable securities": 23800,\n    "Accounts receivable (net)": 205000,\n    "Inventory": 231000,\n    "Total current assets": 515100,\n    "Investments": 64000\n  },\n  "plant_and_equipment": {\n    "Net plant and equipment": 461000,\n    "Less: Accumulated depreciation": 200000\n  },\n  "liabilities_and_stockholders_equity": {\n    "Current liabilities": {\n      "Accounts payable": 93300,\n      "Notes payable": 70000,\n      "Accrued taxes": 16500\n    },\n    "Long-term liabilities": {\n      "Bonds payable": 151200\n    },\n    "Total liabilities": {\n      "Current liabilities": 179800,\n      "Long-term liabilities": 331000\n    },\n    "Stockholders\' equity": {\n      "Preferred stock, $50 par value": 100000,\n      "Common stock, $1 par value": 80000,\n      "Capital paid in excess of par": 190000,\n      "Retained earnings": 339100\n    },\n    "Total stockholders\' equity": 709100,\n    "Total liabilities and stockholders\' equity": 1040100\n  }\n}

''')
j